## Load libraries

In [ ]:
library(keras)
library(dplyr)
library(scales)

## Import dataset
Information about customers consists of 86 variables and includes product usage data and socio-demographic data derived from zip area codes. The data was supplied by the Dutch data mining company Sentient Machine Research and is based on a real world business problem. 
You can download it [here](https://archive.ics.uci.edu/ml/machine-learning-databases/tic-mld/) 

In [ ]:
filename <- "ticdata2000.txt"
df <- read.table(filename,header=FALSE)

## View data
Data dictionary can be seen [here](https://archive.ics.uci.edu/ml/machine-learning-databases/tic-mld/TicDataDescr.txt)

In our use case, we have 85 criteria on users seeing an ad. The last column indicates if they clicked it.

In [ ]:
head(df)

## Split data into trainning and test set

In [ ]:
train <- sample_frac(df,.8)
sid<-as.numeric(rownames(train)) # because rownames() returns character
test<-df[-sid,]

In [ ]:
print(dim(train))
print(dim(test))

## Scale data into range between 0 - 1

In [ ]:
train_scaled <- apply(train,MARGIN = 2, FUN = function(X) rescale(X,to=c(0,1)))
test_scaled <- apply(test,MARGIN = 2, FUN = function(X) rescale(X,to=c(0,1)))

Seperate predictors from target value

In [ ]:
x_train <- train_scaled[,-86]
y_train <- train_scaled[,86]

In [ ]:
x_test <- test_scaled[,-86]
y_test <- test_scaled[,86]

## Build Keras Model

In [ ]:
model <- keras_model_sequential() 
model %>% 
  layer_dense(units = 50, activation = 'relu', input_shape = c(85)) %>% 
  layer_dense(units = 100, activation = 'relu') %>%
  layer_dense(units = 50, activation = 'relu') %>%
  layer_dense(units = 50, activation = 'relu') %>%
  layer_dense(units = 30, activation = 'relu') %>%
  layer_dense(units = 1, activation = 'sigmoid')
model %>% compile(
  loss = 'mean_squared_error',
  optimizer = optimizer_adam(),
  metrics = c('accuracy')
)

In [ ]:
summary(model)

## Train Model

In [ ]:
history <- model %>% fit(
  x_train, y_train, 
  epochs = 10,
  shuffle = TRUE,
  verbose=2,
  validation_data=list(x_test,y_test)
)

### We do not get any output due as the r package runs python in the background and therefore the r console has no output

## Plot history

In [ ]:
plot(history)

## Evaluate Test Set

In [ ]:
test_error_rate <- model %>% evaluate(x_test,y_test,verbose=0)

In [ ]:
test_error_rate